Training, Testing and Deployment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# !pip install sentence-transformers transformers

# import re
# import pandas as pd
# from sentence_transformers import SentenceTransformer, util
# from sklearn.model_selection import train_test_split
# from transformers import DistilBertTokenizer

# # Load pre-trained SentenceTransformer model for embedding generation
# embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# # Load your data
# data = pd.read_csv('/content/classified_documents_with_topics.csv')

# # Preprocessing Function for Text Content
# def preprocess_text(text):
#     text = text.lower()  # Lowercase text
#     text = re.sub(r'http\S+', '', text)  # Remove URLs
#     text = re.sub(r'\d+', '', text)  # Remove numbers
#     text = re.sub(r'\W+', ' ', text)  # Remove non-word characters
#     text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
#     return text.strip()

# # Apply preprocessing to content column
# data['cleaned_content'] = data['Content'].apply(preprocess_text)

# # Generate embeddings for the topic labels
# topic_labels = data['topic_label'].tolist()
# topic_embeddings = embedding_model.encode(topic_labels, convert_to_tensor=True)

# # Define representative sentences for each category
# category_sentences = {'Finance': "Financial trends, market fluctuations, stock regulations, payment compliances, and export reports.",
#     'Geopolitical': "International conflicts, trade disputes, political crises, and country-specific risks.",
#     'Regulatory': "Legal policies, trade regulations, import-export compliance, and government mandates.",
#     'Natural Disasters': "Severe weather events, natural calamities, earthquakes, hurricanes, and supply chain disruptions."}


# # Generate embeddings for each category representative sentence
# category_embeddings = {cat: embedding_model.encode(sent, convert_to_tensor=True)
#                        for cat, sent in category_sentences.items()}

# # Function to categorize topic label based on similarity
# def categorize_topic_label(label_embedding):
#     similarities = {cat: util.cos_sim(label_embedding, emb).item()
#                     for cat, emb in category_embeddings.items()}
#     return max(similarities, key=similarities.get)

# # Map each topic label to a category based on highest similarity
# data['predicted_category'] = [categorize_topic_label(embed) for embed in topic_embeddings]

# # Encode 'predicted_category' to integer labels for training
# category_to_id = {category: idx for idx, category in enumerate(["Finance", "Geopolitical", "Regulatory", "Natural Disasters"])}
# data['label'] = data['predicted_category'].map(category_to_id)

# # Train-test split
# train_texts, val_texts, train_labels, val_labels = train_test_split(
#     data['cleaned_content'].tolist(), data['label'].tolist(),
#     test_size=0.2,
#     random_state=42,
#     stratify=data['label'].tolist()  # Ensures balanced split
# )

# # Tokenizer for BERT-based model (DistilBERT)
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# # Tokenize the texts
# train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
# val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# # Display some of the final DataFrame with mapped categories
# print(data[['topic_label', 'predicted_category', 'cleaned_content']].head())


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


                               topic_label predicted_category  \
0          Financial Compliance and Export            Finance   
1         Trade Regulations and Compliance         Regulatory   
2  Natural Disasters and Financial Reports  Natural Disasters   
3        Geopolitical and Financial Crises       Geopolitical   
4        Geopolitical and Financial Crises       Geopolitical   

                                     cleaned_content  
0  seventh generation sports bold new design powe...  
1  the company stated that issues in battery syst...  
2  next generation core materials strategy to foc...  
3  the transaction is subject to regulatory appro...  
4  volkswagen brand partners with revoltech gmbh ...  


DistilBERT

In [ ]:
import torch
from torch.utils.data import Dataset

class NewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, train_labels)
val_dataset = NewsDataset(val_encodings, val_labels)


In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# Load pre-trained DistilBERT and configure it for classification
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased', num_labels=len(category_to_id)
)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="epoch",     # evaluate each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,0.966848
2,No log,0.779120
3,No log,0.722457


TrainOutput(global_step=303, training_loss=0.9200486790622422, metrics={'train_runtime': 179.9365, 'train_samples_per_second': 13.438, 'train_steps_per_second': 1.684, 'total_flos': 320317594361856.0, 'train_loss': 0.9200486790622422, 'epoch': 3.0})

In [ ]:
# Evaluate
eval_results = trainer.evaluate()
print(f"Validation Accuracy: {eval_results.get('eval_accuracy', eval_results['eval_loss'])}")


Validation Accuracy: 0.7224571108818054


In [ ]:
eval_results

{'eval_loss': 0.7224571108818054,
 'eval_runtime': 3.538,
 'eval_samples_per_second': 57.094,
 'eval_steps_per_second': 7.349,
 'epoch': 3.0}

In [ ]:
def predict_category(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Move inputs to the same device as the model
    for key in inputs:
        inputs[key] = inputs[key].to(model.device)

    outputs = model(**inputs)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    return list(category_to_id.keys())[predicted_label]

# Test the inference function
test_text ="Finance"
print("Predicted Category:", predict_category(test_text))


Predicted Category: Geopolitical


BERT Model: bert-base-uncased

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
# Loading the dataset
data = pd.read_csv('/content/classified_documents_with_topics (1).csv')

In [ ]:
# Preprocessing the data
def preprocess_text(text):
    text = re.sub(r'\W+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

data['cleaned_content'] = data['Content'].apply(preprocess_text)

# Multi-label binary columns for categories
categories = ['Finance', 'Geopolitical', 'Natural Disasters', 'Regulatory']
for category in categories:
    data[category] = data['topic_label'].apply(lambda x: int(category in x))

# Splitting data into train and test sets
X = data['cleaned_content']
y = data[categories]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Defining the Dataset class
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        labels = self.labels.iloc[idx].values
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_length,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.FloatTensor(labels)
        }

# Loading BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Creating DataLoaders
train_dataset = NewsDataset(X_train, y_train, tokenizer)
test_dataset = NewsDataset(X_test, y_test, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

In [ ]:
# Defining the model
class BertMultiLabelClassifier(nn.Module):
    def __init__(self, num_labels):
        super(BertMultiLabelClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        output = self.dropout(pooled_output)
        return self.out(output)

# Initialize model, optimizer, and loss function
model = BertMultiLabelClassifier(num_labels=len(categories))
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
criterion = nn.BCEWithLogitsLoss()

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertMultiLabelClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
# Training the model
def train_epoch(model, data_loader, criterion, optimizer):
    model.train()
    losses = []
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
    return sum(losses) / len(losses)

In [ ]:
# Evaluation
def eval_model(model, data_loader, criterion):
    model.eval()
    y_true = []
    y_pred = []
    losses = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)
            losses.append(loss.item())

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(torch.sigmoid(outputs).cpu().numpy())

    y_pred = [[1 if pred >= 0.5 else 0 for pred in preds] for preds in y_pred]
    f1 = f1_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)
    return sum(losses) / len(losses), accuracy, f1

In [ ]:
# # Training loop for 10 epochs
# num_epochs = 10
# for epoch in range(num_epochs):
#     train_loss = train_epoch(model, train_loader, criterion, optimizer)
#     val_loss, val_accuracy, val_f1 = eval_model(model, test_loader, criterion)

#     print(f"Epoch {epoch + 1}/{num_epochs}")
#     print(f"Train Loss: {train_loss}")
#     print(f"Val Loss: {val_loss}, Val Accuracy: {val_accuracy}, Val F1 Score: {val_f1}")

In [ ]:
# Training loop
num_epochs = 15
for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_accuracy, val_f1 = eval_model(model, test_loader, criterion)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_loss}")
    print(f"Val Loss: {val_loss}, Val Accuracy: {val_accuracy}, Val F1 Score: {val_f1}")

Epoch 1/15
Train Loss: 0.45865881873007436
Val Loss: 0.39462414939029544, Val Accuracy: 0.48507462686567165, Val F1 Score: 0.9162507481271773
Epoch 2/15
Train Loss: 0.3441802667296348
Val Loss: 0.2859119556698145, Val Accuracy: 0.6243781094527363, Val F1 Score: 0.932846928610694
Epoch 3/15
Train Loss: 0.22551997728757003
Val Loss: 0.24237303406584496, Val Accuracy: 0.6691542288557214, Val F1 Score: 0.9414025628788427
Epoch 4/15
Train Loss: 0.13976639395218288
Val Loss: 0.22659879141286307, Val Accuracy: 0.6915422885572139, Val F1 Score: 0.9483727653573175
Epoch 5/15
Train Loss: 0.07853573357769802
Val Loss: 0.23129808668996774, Val Accuracy: 0.7089552238805971, Val F1 Score: 0.9523089344606277
Epoch 6/15
Train Loss: 0.05391641467605805
Val Loss: 0.2317663510786552, Val Accuracy: 0.7263681592039801, Val F1 Score: 0.9511904973093407
Epoch 7/15
Train Loss: 0.030819118426028472
Val Loss: 0.24599466774174394, Val Accuracy: 0.7313432835820896, Val F1 Score: 0.9510583596700645
Epoch 8/15
Trai

Saving the trained model

In [ ]:
# # Save the model
# torch.save(model.state_dict(), 'bert_multi_label_model_15ep.pth')

Testing the saved model

In [ ]:
import torch
from transformers import BertTokenizer
import torch.nn as nn

# Loading the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertMultiLabelClassifier(num_labels=len(categories))
model.load_state_dict(torch.load('/content/drive/MyDrive/bert_model_10epochs.pth'))
model.eval()  #evaluation mode

# Move model to device if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Function to preprocess and tokenize new text
def preprocess_and_predict(text, threshold=0.7):  # Set threshold to 0.7
    # Preprocessing and tokenizing the text as before
    text = re.sub(r'\W+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip().lower()

    encoding = tokenizer.encode_plus(
        text,
        max_length=512,
        add_special_tokens=True,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        output = model(input_ids=input_ids, attention_mask=attention_mask)
        probs = torch.sigmoid(output).cpu().numpy()

    # Applying a higher threshold to be more selective
    predicted_labels = [categories[i] for i, prob in enumerate(probs[0]) if prob >= threshold]

    return predicted_labels


# Test on new texts
new_texts = [
    '''AROMAS, Calif. (KION-TV) Friday evening at 4:38 p.m., an earthquake registering 2.9 shook homes near Aromas.

The quake's epicenter was 3 miles SSE from Aromas, with a depth of 4.8 miles, according to the U.S. Geological Survey (USGS).

There have been reports of people feeling the quake in Prunedale and Watsonville.

''',
  '''Ford Trucks commissions ZF to supply electric drives of the latest generation for its upcoming line of electrified trucks; the first product will be ZFâ€™s central electric drive, the CeTrax 2 dual, followed by its Electrically Powered Hydraulic Steering Pump (EPHS)
ZFâ€™s Commercial Vehicle Solutions (CVS) division has further underlined its status as Europeâ€™s leading producer of electric drives after securing a long-term supplier agreement with Ford Trucks for e-mobility technologies. The agreement will see Fordâ€™s only heavy commercial vehicle brand, Ford Trucks, equip its upcoming lines of electrified heavy trucks with ZFâ€™s latest state-of-the-art zero emissions technologies.
â€œWith Ford Trucks, we have gained a major international customer for our new e-mobility drive platform. This underlines our leading position in Europe for electric commercial drives,â€ said Prof. Dr. Peter Laier, Member of the ZF Board of Management and responsible for the Commercial Vehicle Solutions Division. â€œFord Trucksâ€™ trust in us shows that we can offer market-leading solutions for e-mobility to support our customers in their transformation towards sustainable transportation solutions.â€
Emrah Duman, Ford Trucksâ€™ vice-president, said: â€œBeyond innovative processes, product readiness and competitiveness, a trusted relationship and the highest quality standards are essential for us. Additionally, a well-defined sustainability roadmap is key for our selected suppliers supporting us in our mission to develop best-in-class commercial vehicles that enable sustainable transport solutions. And we are glad that a long-established brand like ZF is one of them. Ford Trucks has always been at the forefront of innovation in the heavy commercial vehicle industry, and this new agreement with ZF further solidifies its mission to lead the shift towards sustainable transport.â€
The first product to be supplied by ZF will be its central electric drive, the CeTrax 2 dual.
ZFâ€™s CeTrax 2 series: compact and modular design
ZF will first supply the CeTrax 2 dual electric central drive for Ford Trucksâ€™ heavy duty Electric Rigid Truck, which is making its world premiere at the IAA Transportation 2024. Under the agreement, ZF will also supply further electric drive variants as upcoming models enter production.
The compact and modular design of ZFâ€™s CeTrax 2 series enables the drives to fit within the same mounting space as a conventional ICE transmission. This gives manufacturers the flexibility to install either a combustion drivetrain or a CeTrax 2 electric central drive on the same production line, depending on the customerâ€™s configuration. This flexibility enables manufacturers to manage the transition towards e-mobility production effectively, especially during the initial ramp up phase when volumes are relatively small compared to traditional ICE lines.
As part of the agreement, ZF will also equip Ford Trucksâ€™ Electric Rigid Truck with its Electrically Powered Hydraulic Steering Pump (EPHS). Equipped with an e-motor, this steering device is designed to provide on-demand hydraulic flow enabling the efficient operation of the commercial vehicleâ€™s hydraulic steering system independent of the vehicleâ€™s drivetrain.


  ''',
]

for text in new_texts:
    predicted_labels = preprocess_and_predict(text)
    print(f"Text: {text}")
    print(f"Predicted Categories: {predicted_labels}\n")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-47-d1c816eaf550>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unles

Text: AROMAS, Calif. (KION-TV) Friday evening at 4:38 p.m., an earthquake registering 2.9 shook homes near Aromas.

The quake's epicenter was 3 miles SSE from Aromas, with a depth of 4.8 miles, according to the U.S. Geological Survey (USGS).

There have been reports of people feeling the quake in Prunedale and Watsonville.


Predicted Categories: ['Finance', 'Geopolitical', 'Natural Disaster', 'Regulatory']

Text: Ford Trucks commissions ZF to supply electric drives of the latest generation for its upcoming line of electrified trucks; the first product will be ZFâ€™s central electric drive, the CeTrax 2 dual, followed by its Electrically Powered Hydraulic Steering Pump (EPHS)
ZFâ€™s Commercial Vehicle Solutions (CVS) division has further underlined its status as Europeâ€™s leading producer of electric drives after securing a long-term supplier agreement with Ford Trucks for e-mobility technologies. The agreement will see Fordâ€™s only heavy commercial vehicle brand, Ford Trucks, equip 

In [ ]:
val_accuracy

0.736318407960199

Accuracy for model trained on 10 epochs

In [ ]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from torch.utils.data import DataLoader

# Loading the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertMultiLabelClassifier(num_labels=len(categories))
model.load_state_dict(torch.load('/content/drive/MyDrive/Copy of bert_model_10epochs.pth'))
#model.load_state_dict(torch.load('/content/drive/MyDrive/bert_model_10epochs.pth', map_location=torch.device('cpu')))
model.eval()  #evaluation mode

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Preparing test data (assuming you already split the data and created a test DataLoader)
test_dataset = NewsDataset(X_test, y_test, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8)

# Function to calculate metrics for multi-label classification
def calculate_metrics(y_true, y_pred, threshold=0.5):
    y_pred = (y_pred >= threshold).astype(int)  # Convert probabilities to binary predictions
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    return accuracy, f1, precision, recall

# Evaluating the model on the test set
y_true = []
y_pred = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].cpu().numpy()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probabilities = torch.sigmoid(outputs).cpu().numpy()  # Get probabilities

        y_true.extend(labels)
        y_pred.extend(probabilities)

# Calculating accuracy and other metrics
y_true = np.array(y_true)
y_pred = np.array(y_pred)

accuracy, f1, precision, recall = calculate_metrics(y_true, y_pred, threshold=0.5)
print("Results for bert model trained for 10 epochs")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")


<ipython-input-14-0c795533d703>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/Copy of bert_model_10epochs.pth'))


Results for bert model trained for 10 epochs
Accuracy: 0.7661691542288557
F1 Score: 0.9612355122085545
Precision: 0.9629960747942654
Recall: 0.959499263622975


Accuracy for model trained on 15 epochs

In [ ]:
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from torch.utils.data import DataLoader

# Loading the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertMultiLabelClassifier(num_labels=len(categories))
model.load_state_dict(torch.load('/content/drive/MyDrive/bert_model_15epochs.pth'))
model.eval()  # Set model to evaluation mode

# Moving model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Preparing test data
test_dataset = NewsDataset(X_test, y_test, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8)

# Function to calculate metrics for multi-label classification
def calculate_metrics(y_true, y_pred, threshold=0.5):
    y_pred = (y_pred >= threshold).astype(int)  # Convert probabilities to binary predictions
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    return accuracy, f1, precision, recall

# Evaluating the model on the test set
y_true = []
y_pred = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].cpu().numpy()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probabilities = torch.sigmoid(outputs).cpu().numpy()  # Get probabilities

        y_true.extend(labels)
        y_pred.extend(probabilities)

# Calculating accuracy and other metrics
y_true = np.array(y_true)
y_pred = np.array(y_pred)

accuracy, f1, precision, recall = calculate_metrics(y_true, y_pred, threshold=0.5)
print("Results for bert model trained for 15 epochs")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-19-e7c6cefec608>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unles

Results for bert model trained for 15 epochs
Accuracy: 0.736318407960199
F1 Score: 0.9524991514052593
Precision: 0.9463558144731846
Recall: 0.9587628865979382


Classification report for model trained on 10 epochs

In [ ]:
import torch
import numpy as np
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader
from transformers import BertTokenizer

# Loading the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertMultiLabelClassifier(num_labels=len(categories))
model.load_state_dict(torch.load('/content/drive/MyDrive/bert_model_10epochs.pth'))
model.eval()  # Set model to evaluation mode

# Moving model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Preparing test data (assuming you already split the data and created a test DataLoader)
test_dataset = NewsDataset(X_test, y_test, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8)

# Evaluating the model on the test set
y_true = []
y_pred = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].cpu().numpy()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probabilities = torch.sigmoid(outputs).cpu().numpy()  # Get probabilities

        y_true.extend(labels)
        y_pred.extend(probabilities)

# Converting predictions to binary (0 or 1) based on threshold
y_pred = np.array(y_pred) >= 0.5
y_true = np.array(y_true)

# Generating classification report
report = classification_report(y_true, y_pred, target_names=categories)
print("Classification Report for BERT Model Trained for 10 Epochs")
print(report)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-17-f71d85cf8793>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unle

Classification Report for BERT Model Trained for 10 Epochs
                   precision    recall  f1-score   support

          Finance       0.96      0.95      0.95       333
     Geopolitical       0.95      0.95      0.95       330
Natural Disasters       0.96      0.95      0.96       329
       Regulatory       0.98      0.99      0.98       366

        micro avg       0.96      0.96      0.96      1358
        macro avg       0.96      0.96      0.96      1358
     weighted avg       0.96      0.96      0.96      1358
      samples avg       0.97      0.97      0.96      1358



Classification report for model trained on 15 epochs

In [ ]:
import torch
import numpy as np
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader
from transformers import BertTokenizer

# Loading the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertMultiLabelClassifier(num_labels=len(categories))
model.load_state_dict(torch.load('/content/drive/MyDrive/Copy of bert_model_15epochs.pth'))
model.eval()  # Set model to evaluation mode

# Moving model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Preparing test data
test_dataset = NewsDataset(X_test, y_test, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8)

# Evaluating the model on the test set
y_true = []
y_pred = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].cpu().numpy()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probabilities = torch.sigmoid(outputs).cpu().numpy()  # Get probabilities

        y_true.extend(labels)
        y_pred.extend(probabilities)

# Converting predictions to binary (0 or 1) based on threshold
y_pred = np.array(y_pred) >= 0.5
y_true = np.array(y_true)

# Generating classification report
report = classification_report(y_true, y_pred, target_names=categories)
print("Classification Report for BERT Model Trained for 15 Epochs")
print(report)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-18-1d8d47dee87a>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unle

Classification Report for BERT Model Trained for 15 Epochs
                   precision    recall  f1-score   support

          Finance       0.94      0.96      0.95       333
     Geopolitical       0.92      0.95      0.93       330
Natural Disasters       0.95      0.95      0.95       329
       Regulatory       0.98      0.98      0.98       366

        micro avg       0.95      0.96      0.95      1358
        macro avg       0.95      0.96      0.95      1358
     weighted avg       0.95      0.96      0.95      1358
      samples avg       0.95      0.96      0.95      1358



Gradio deployment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 125.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:
categories = ['Finance', 'Geopolitical', 'Natural Disasters', 'Regulatory']

In [ ]:
class BertMultiLabelClassifier(nn.Module):
    def __init__(self, num_labels):
        super(BertMultiLabelClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        output = self.dropout(pooled_output)
        return self.out(output)

# Initializing model, optimizer, and loss function
model = BertMultiLabelClassifier(num_labels=len(categories))
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
criterion = nn.BCEWithLogitsLoss()

# Moving model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertMultiLabelClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
import gradio as gr
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn

# Defining the categories
categories = ['Finance', 'Geopolitical', 'Natural Disasters', 'Regulatory']
# Loading trained model weights
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model.load_state_dict(torch.load('/content/drive/MyDrive/bert_model_10epochs.pth', map_location=device))
model.eval()

# Defining a prediction function
def predict(text):
    # Preprocessing and tokenizing input text
    encoding = tokenizer.encode_plus(
        text,
        max_length=512,
        add_special_tokens=True,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Making prediction
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probs = torch.sigmoid(outputs).cpu().numpy()[0]

    # Converting probabilities to labels
    results = {category: f"{prob:.2f}" for category, prob in zip(categories, probs)}
    return results


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-7-4f4616bf3633>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unles

In [ ]:
interface = gr.Interface(
    fn=predict,
    inputs=[
        gr.Textbox(lines=6, placeholder="Type or paste your news content here...", label="News Content"),
    ],
    outputs=gr.Label(num_top_classes=len(categories), label="Category Probabilities"),
    title="Supplier News Categorization",
    description=(
        "Enter news content, and this tool will classify it into multiple categories: "
        "Finance, Geopolitical, Natural Disasters, and Regulatory. "
        "Each category is given a probability score indicating how likely it is that the news "
        "content belongs to that topic. "
    ),
    examples=[
        ["New regulations on cryptocurrency trading are being discussed by the financial authorities."],
        ["A powerful earthquake has struck the coastal region, causing widespread damage."],
        ["The ongoing trade negotiations between major economies have raised geopolitical concerns."],
        ["The central bank announced changes in the interest rate policies affecting the financial sector."],
    ],
    flagging_mode="never",  # Updated flagging parameter
    theme="default",  # Set theme to default (light mode)
)

# Interface launch with debugging enabled in Colab
interface.launch(debug=True, share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a7535b1957472ea1fd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a7535b1957472ea1fd.gradio.live
